# Import Modules

## Standard

In [1]:
import sys
import os
import os.path as path
import xarray as xr
from netCDF4 import Dataset
import numpy as np
import pandas as pd
from scipy import interpolate
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

/g/g92/jha3/VirtualEnv/py3_ml_wind/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


## User Defined

In [2]:
sys.path.insert(0, os.path.dirname(os.getcwd()))
from data_processing_slice import *
from helper_GAD import *

# Paths

In [3]:
WRF_result_base_loc ='/p/lustre2/jha3/fromWill/TurbTest/NREL_5MW'
GAD_param_output_loc = os.path.join(WRF_result_base_loc, 'GAD_Param_Study_Output')
GAD_param_power_curve_loc = os.path.join(GAD_param_output_loc, 'PowerCurve')
GAD_param_slice_loc = os.path.join(GAD_param_output_loc, 'Slice')
os.system('mkdir -p %s'%GAD_param_power_curve_loc)
os.system('mkdir -p %s'%GAD_param_slice_loc)

0

# Flags

In [4]:
savefig = True
induction_effect = False
grid_effect = True
epsilon_effect = False

# Common Stuff

In [5]:
vhub = [3, 4.5, 6, 7.5, 9, 10.5, 12, 13.5, 15]
#vhub = [3, 4.5, 6, 7.5, 9, 10.5, 13.5, 15, 18]
ind1_for_tab = 3 # To be used in the paper
ind2_for_tab = 4 # To be used in the paper
ind3_for_tab = 5 # To be used in the paper
outfile = 'wrfout_d01_0001-01-01_00:00:00'
tsoutfile = 'tsout_d01_0001-01-01_00:00:00'

# Effect of Induction Type

In [6]:
if induction_effect:
	figFileName = os.path.join(GAD_param_power_curve_loc, 'power_induction.png')
	tab_data_file = os.path.join(GAD_param_power_curve_loc, 'power_induction.csv')
	plt_title   = '$Grid: \Delta x = \Delta y = 4 m, \Delta z = 4 m; Gaussian: \epsilon/\Delta_{grid} = 1.00 $'
	case_dir_map = {'Glauert_Tip_dx_04_dz_04_an_fixed_at_iter_eps_1.00': {'legend':'Glauert, a$_{n}$ = 0.07, a$_{t}$ = iterated', 'line_style': 'b'},
		        'Glauert_Tip_dx_04_dz_04_an_iter_at_iter_eps_1.00': {'legend':'Glauert, a$_{n}$ = iterated, a$_{t}$ = iterated', 'line_style': 'g'},
			'Shen_Tip_dx_04_dz_04_an_fixed_at_iter_eps_1.00': {'legend':'Shen, a$_{n}$ = 0.07, a$_{t}$ = iterated', 'line_style': 'r--'},
			'Shen_Tip_dx_04_dz_04_an_iter_at_iter_eps_1.00': {'legend':'Shen, a$_{n}$ = iterated, a$_{t}$ = iterated', 'line_style': 'm--'},
			'Tip_dx_04_dz_04_an_0.0_at_0.0_eps_1.00': {'legend':'a$_{n}$ = 0.0, a$_{t}$ = 0.0', 'line_style': 'c'}}

# Effect of Grid Resolution

In [7]:
if grid_effect:
	figFileName = os.path.join(GAD_param_power_curve_loc, 'power_grid_effect.png')
	tab_data_file = os.path.join(GAD_param_power_curve_loc, 'power_grid_effect.csv')
	plt_title   = '$ Induction: Glauert, a_{n} = 0.07, a_{t} = iterated; Gaussian: \epsilon/\Delta_{grid} = 1.00 $'
	case_dir_map = {'Glauert_Tip_dx_04_dz_04_an_fixed_at_iter_eps_1.00': {'legend':'$\Delta x = \Delta y = 4 m, \Delta z = 4 m $', 'line_style': 'b'},
			'Glauert_Tip_dx_08_dz_08_an_fixed_at_iter_eps_1.00': {'legend':'$\Delta x = \Delta y = 8 m, \Delta z = 8 m $', 'line_style': 'g'},
			'Glauert_Tip_dx_16_dz_04_an_fixed_at_iter_eps_1.00': {'legend':'$\Delta x = \Delta y = 16 m, \Delta z = 4 m $', 'line_style': 'r--'},
			'Glauert_Tip_dx_16_dz_16_an_fixed_at_iter_eps_1.00': {'legend':'$\Delta x = \Delta y = 16 m, \Delta z = 16 m $', 'line_style': 'm--'}}

# Effect of Gaussian Spreading

In [8]:
if epsilon_effect:
	figFileName = os.path.join(GAD_param_power_curve_loc, 'power_gaussian_effect.png')
	tab_data_file = os.path.join(GAD_param_power_curve_loc, 'power_gaussian_effect.csv')
	plt_title   = '$Grid: \Delta x = \Delta y = 4 m, \Delta z = 4 ; Induction: Glauert, a_{n} = 0.07, a_{t} = iterated $'
	case_dir_map = {'Glauert_Tip_dx_04_dz_04_an_fixed_at_iter_eps_0.707': {'legend':'$ \epsilon/\Delta_{grid} = 1/\sqrt{2} $', 'line_style': 'b'},
			'Glauert_Tip_dx_04_dz_04_an_fixed_at_iter_eps_1.00': {'legend':'$ \epsilon/\Delta_{grid} = 1.00$', 'line_style': 'g'},
			'Glauert_Tip_dx_04_dz_04_an_fixed_at_iter_eps_1.4142': {'legend':'$ \epsilon/\Delta_{grid} = \sqrt{2} $', 'line_style': 'r--'},
			'Glauert_Tip_dx_04_dz_04_an_fixed_at_iter_eps_2.00': {'legend':'$ \epsilon/\Delta_{grid} = 2.00 $', 'line_style': 'm--'}}

# Cases of Interest

In [9]:
case_keys = list(case_dir_map.keys())
case_keys.sort()
print ('Cases: \n {}'.format(case_keys))
#case_keys

Cases: 
 ['Glauert_Tip_dx_04_dz_04_an_fixed_at_iter_eps_1.00', 'Glauert_Tip_dx_08_dz_08_an_fixed_at_iter_eps_1.00', 'Glauert_Tip_dx_16_dz_04_an_fixed_at_iter_eps_1.00', 'Glauert_Tip_dx_16_dz_16_an_fixed_at_iter_eps_1.00']


# NREL Data

In [ ]:
NREL_Data, ws_NREL, power_NREL, power_interp = prepare_NREL_data (WRF_result_base_loc, vhub)

# Initialize Tabulated Data

In [ ]:
case_tab, power1_tab, power2_tab, power3_tab, \
error1_tab, error2_tab, error3_tab = \
                initialize_tabulated_data (power_interp, \
                                           ind1_for_tab, ind2_for_tab, ind3_for_tab)

# Read the Power Data for Cases of Interest

In [ ]:
print ('Cases under consideration : \n')
for case in case_keys:
    case_dir_map, case_tab, \
    power1_tab, power2_tab, power3_tab, \
    error1_tab, error2_tab, error3_tab = \
    read_power_for_a_case (WRF_result_base_loc, case, vhub, power_interp, outfile, \
                           case_dir_map, case_tab, \
                           power1_tab, power2_tab, power3_tab, \
                           error1_tab, error2_tab, error3_tab, \
                           ind1_for_tab, ind2_for_tab, ind3_for_tab)

# Prepare Tabulated Data for a Few Wind Speeds

In [ ]:
tabulate_data_for_cases (tab_data_file, vhub, case_tab, \
                         power1_tab, power2_tab, power3_tab, \
                         error1_tab, error2_tab, error3_tab, \
                         ind1_for_tab, ind2_for_tab, ind3_for_tab)

# Create Power Curve Plots

## Power Curve

In [ ]:
plot_power_curve_for_cases (NREL_Data, vhub, case_keys, case_dir_map, plt_title, \
                            figFileName, savefig)

## Error in Power Curve

In [ ]:
plot_power_curve_error_for_cases (vhub, case_keys, case_dir_map, plt_title)

# Read Slice Data

In [10]:
case = case_keys[0]
ws = 9
z_ind = 22
time_ind = 30

In [11]:
case_ws_wrf_data = read_wrfout_data_for_case_ws (WRF_result_base_loc, case, ws, outfile)
case_ws_ts_data = read_tsout_data_for_case_ws (WRF_result_base_loc, case, ws, tsoutfile)

/g/g92/jha3/VirtualEnv/py3_ml_wind/lib/python3.7/site-packages/xarray/coding/times.py:526: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)


In [18]:
slice_data_UTS = extract_slices_from_tsout_file (case_ws_ts_data, 'UTS', z_ind)
slice_data_UTS = extract_slices_from_tsout_file (case_ws_ts_data, 'UTS', z_ind, time_ind)

In [19]:
slice_data_UTS

array([[8.420158 , 8.420105 , 8.420065 , ..., 8.408976 , 8.409105 ,
        8.409239 ],
       [8.413488 , 8.413443 , 8.413387 , ..., 8.4105425, 8.410704 ,
        8.410849 ],
       [8.40689  , 8.406841 , 8.406792 , ..., 8.412224 , 8.412405 ,
        8.412569 ],
       ...,
       [8.481346 , 8.481367 , 8.481394 , ..., 8.483938 , 8.4839   ,
        8.483847 ],
       [8.481432 , 8.481453 , 8.481476 , ..., 8.483894 , 8.483852 ,
        8.483801 ],
       [8.48147  , 8.481493 , 8.481515 , ..., 8.483878 , 8.483835 ,
        8.48378  ]], dtype=float32)

# Slice Plots

In [ ]:
case_ws_wrf_data.dims

In [ ]:
np.array(case_ws_wrf_data['PHB'].isel(south_north = 128).isel(west_east = 128).isel(Time=1))[23]/9.81

In [ ]:
case_ws_ts_data.dims

In [20]:
np.array(case_ws_ts_data['VTS'].isel(Time=30).isel(bottom_top = 22)).shape

(256, 256)

In [22]:
#case_ws_ts_data